# Unicode Problems & Answers

> This notebook documents answers to specific Unicode questions as part of the project.

## Problem (unicode1): Understanding Unicode

This section addresses questions related to the basic understanding of Unicode characters, specifically the null character.

**(a) What does `chr(0)` return?**

`chr(0)` returns the null character, which is represented as `\x00` in Python string literals. The null character is a control character with Unicode code point U+0000.

In [1]:
null_char = chr(0)
print(f"Type: {type(null_char)}, Value: {null_char!r}")

Type: <class 'str'>, Value: '\x00'


**(b) What is its string representation `__repr__()` vs `print()` output?**

- The string representation `__repr__()` of `chr(0)` is `'\x00'`. This is an unambiguous representation of the string containing the null character.
- The `print()` output of `chr(0)` is typically invisible or terminal-dependent. Most terminals do not render a visible glyph for the null character. Some terminals might ignore it, while others might display a placeholder or interpret it as a string terminator.

In [2]:
null_char = chr(0)
print(f"repr(null_char): {repr(null_char)}")
print("Output of print(null_char) (might be invisible):")
print(null_char)
print("Done printing null_char.")

repr(null_char): '\x00'
Output of print(null_char) (might be invisible):
 
Done printing null_char.


**(c) Is the null character part of the string's data or just a rendering artifact?**

The null character is definitively part of the string's data. It is stored in memory as part of the string. Its rendering (or lack thereof) is application-dependent (e.g., how a terminal, text editor, or browser chooses to display it). In some contexts (like C-style strings), the null character acts as a terminator, but in Python strings, it's just another character, and the string's length includes it.

In [3]:
s_with_null = "hello\x00world"
print(f"String: {s_with_null!r}")
print(f"Length: {len(s_with_null)}") # Length will be 11 (5 + 1 for null + 5)
print(f"'\x00' in s_with_null: {'\x00' in s_with_null}")

String: 'hello\x00world'
Length: 11
' ' in s_with_null: True


## Problem (unicode2): Unicode Encodings

This section addresses questions related to Unicode encodings, particularly UTF-8.

**(a) Why is UTF-8 preferred as a text encoding?**

UTF-8 is widely preferred for several reasons:
1.  **ASCII Compatibility:** UTF-8 is backward compatible with ASCII. The first 128 Unicode characters (U+0000 to U+007F) are encoded as a single byte in UTF-8, identical to their ASCII representation. This means ASCII text is also valid UTF-8 text.
2.  **Variable-Width Efficiency:** UTF-8 uses a variable number of bytes per character (1 to 4 bytes). Common characters, especially those in the Latin script, are encoded using fewer bytes, making it relatively space-efficient for texts predominantly using these characters. More complex characters use more bytes.
3.  **Web Standard:** It is the dominant encoding for the World Wide Web, ensuring broad compatibility across different systems and browsers.
4.  **Self-Synchronizing:** It's possible to determine the start of a character sequence from any point in a byte stream, which helps in error recovery.
5.  **No Null Bytes (for non-null characters):** Unlike UTF-16 or UTF-32 which can have null bytes as part of a multi-byte character representation (e.g. for 'A' in UTF-16BE is `\x00\x41`), UTF-8 only uses the null byte `\x00` to represent the null character U+0000. This avoids issues with C-style string functions that treat null bytes as terminators.
6.  **Universal Character Set Support:** It can represent every character in the Unicode standard.

**(b) Why is `decode_utf8_bytes_to_str_wrong` incorrect? Provide an example.**

The hypothetical function `decode_utf8_bytes_to_str_wrong` (which attempts to decode individual bytes as if each byte is a complete UTF-8 character) is incorrect because UTF-8 is a variable-width encoding. Many characters are represented by sequences of multiple bytes (2, 3, or 4 bytes). Decoding each byte individually will fail for these multi-byte characters because a single byte from a multi-byte sequence is not a valid UTF-8 character on its own (unless it's an ASCII character).

**Example:**
The character 'é' (Latin small letter e with acute, U+00E9) is encoded in UTF-8 as the two-byte sequence `b'\xc3\xa9'`.

If `decode_utf8_bytes_to_str_wrong` processes `b'\xc3'` and `b'\xa9'` separately:
-   `b'\xc3'.decode('utf-8')` would raise a `UnicodeDecodeError` because `\xc3` is an invalid start byte for a single-byte UTF-8 character and is incomplete as a multi-byte sequence.
-   Similarly, `b'\xa9'.decode('utf-8')` would also raise a `UnicodeDecodeError` because `\xa9` is a continuation byte and cannot start a UTF-8 sequence.

The correct way is to decode the entire byte sequence together: `b'\xc3\xa9'.decode('utf-8')` which correctly yields `'é'`.

In [4]:
char_bytes = b'\xc3\xa9' # UTF-8 for 'é'
print(f"Correct decoding: {char_bytes.decode('utf-8')}")

print("\nAttempting to decode individual bytes (will cause errors):")
byte1 = char_bytes[0:1] # b'\xc3'
byte2 = char_bytes[1:2] # b'\xa9'

try:
    print(f"Decoding byte1 ({byte1!r}): {byte1.decode('utf-8')}")
except UnicodeDecodeError as e:
    print(f"Error decoding byte1 ({byte1!r}): {e}")

try:
    print(f"Decoding byte2 ({byte2!r}): {byte2.decode('utf-8')}")
except UnicodeDecodeError as e:
    print(f"Error decoding byte2 ({byte2!r}): {e}")

Correct decoding: é

Attempting to decode individual bytes (will cause errors):
Error decoding byte1 (b'\xc3'): 'utf-8' codec can't decode byte 0xc3 in position 0: unexpected end of data
Error decoding byte2 (b'\xa9'): 'utf-8' codec can't decode byte 0xa9 in position 0: invalid start byte


**(c) Provide an invalid UTF-8 two-byte sequence and explain why it's invalid.**

**Example 1: Overlong Encoding `b'\xc0\xaf'`**

-   **Sequence:** `b'\xc0\xaf'`
-   **Why it's invalid:** This is an example of an "overlong encoding." The byte `\xc0` indicates the start of a 2-byte sequence. However, UTF-8 rules state that characters that can be represented with a smaller number of bytes *must* be encoded using that smaller number. The sequence `b'\xc0\xaf'` would decode to U+002F ('/'), which should be encoded as the single byte `b'\x2f'`. UTF-8 decoders must reject overlong encodings as they can be a security risk (e.g., bypassing filters that look for specific byte patterns).

**Example 2: Invalid Continuation Byte / Unexpected Start Byte `b'\xc3\x20'`**

- **Sequence:** `b'\xc3\x20'`
- **Why it's invalid:** The byte `\xc3` indicates the start of a 2-byte sequence. The following byte must be a continuation byte, meaning it must be in the range `\x80` to `\xbf` (i.e., its first two bits are `10`). The byte `\x20` (ASCII space) is `00100000` in binary. Since its first bit is `0`, it's an ASCII character and cannot be a continuation byte for `\xc3`. A UTF-8 decoder encountering `\xc3` followed by `\x20` would recognize this as an invalid sequence.

**Example 3: Surrogate Half `b'\xed\xa0\x80'` (this is 3 bytes, but shows another type of invalidity related to specific ranges)**

Although the question asks for a two-byte sequence, it's useful to know other invalidities.
- **Sequence:** `b'\xed\xa0\x80'`
- **Why it's invalid:** This sequence attempts to encode U+D800. Unicode code points from U+D800 to U+DFFF are reserved for UTF-16 surrogate pairs and are not valid Unicode characters themselves. UTF-8 must not encode these code points.


In [5]:
invalid_seq_overlong = b'\xc0\xaf'
try:
    print(f"Decoding invalid_seq_overlong ({invalid_seq_overlong!r}): {invalid_seq_overlong.decode('utf-8')}")
except UnicodeDecodeError as e:
    print(f"Error decoding invalid_seq_overlong ({invalid_seq_overlong!r}): {e}")

invalid_seq_bad_continuation = b'\xc3\x20' 
try:
    print(f"Decoding invalid_seq_bad_continuation ({invalid_seq_bad_continuation!r}): {invalid_seq_bad_continuation.decode('utf-8')}")
except UnicodeDecodeError as e:
    print(f"Error decoding invalid_seq_bad_continuation ({invalid_seq_bad_continuation!r}): {e}")

invalid_seq_surrogate = b'\xed\xa0\x80'
try:
    print(f"Decoding invalid_seq_surrogate ({invalid_seq_surrogate!r}): {invalid_seq_surrogate.decode('utf-8')}")
except UnicodeDecodeError as e:
    print(f"Error decoding invalid_seq_surrogate ({invalid_seq_surrogate!r}): {e}")

Error decoding invalid_seq_overlong (b'\xc0\xaf'): 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
Error decoding invalid_seq_bad_continuation (b'\xc3 '): 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte
Error decoding invalid_seq_surrogate (b'\xed\xa0\x80'): 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
